In [26]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from random import randint

In [27]:
INPUT_DATA_PATH = 'test_input.npy'
OUTPUT_DATA_PATH = 'test_output.npy'
x_input = np.load(INPUT_DATA_PATH)
print (x_input.shape)
y_input = np.load(OUTPUT_DATA_PATH) #np.array([randint(0, 3) for _ in range(len(x_input))])
#np.load(OUTPUT_DATA_PATH)
print (y_input.shape)

(20, 576, 576, 3)
(20, 576, 576, 3)


In [28]:
SPLIT_IDX = 15 #750
x_train = torch.tensor(x_input[:SPLIT_IDX], dtype=torch.float)
print (x_train.shape)
x_train = x_train.permute(0, 3, 1, 2) # from NHWC to NCHW
print (x_train.shape)
x_test = torch.tensor(x_input[SPLIT_IDX:], dtype=torch.float).permute(0, 3, 1, 2)

y_train = torch.tensor(y_input[:SPLIT_IDX], dtype=torch.float).permute(0, 3, 1, 2)
y_test = torch.tensor(y_input[SPLIT_IDX:], dtype=torch.float).permute(0, 3, 1, 2)

class LinesDataset(TensorDataset):
        def __init__(self, lines, labels):
                self.lines = lines
                self.labels = labels
        def __len__(self):
                return len(self.lines)
        def __getitem__(self, idx):
                line = torch.tensor(self.lines[idx])
                label = torch.tensor(self.labels[idx])
                sample = {"Line": line, "Label": label}
                return sample


train_dataset = TensorDataset(x_train,y_train) # create your datset
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True) # create your dataloader

print (len(train_dataloader))

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True)


torch.Size([15, 576, 576, 3])
torch.Size([15, 3, 576, 576])
3


In [29]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self, input_channels, input_size):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=1, kernel_size=(9,9), stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            # convolutional layers + activation functions and pooling layers in between
            # nn.Conv2d(in_channels=input_channels, out_channels=1, kernel_size=(9,9), stride=(1,1))
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [30]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
        print(X.shape)
        pred = model(X)
        # print(f"predicted: {pred}")
        # # print(f"actual: {y}")
        loss =  F.nll_loss(pred, y)
        #loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        
        print(f"loss: {loss:>7f}")

        # if batch % 50 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
model = NeuralNetwork(3, (576, 576))
learning_rate = 0.001
batch_size = 5
epochs = 5
# Initialize the loss function
# loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss() # can change this to another loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [32]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # test_loop(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
torch.Size([5, 3, 576, 576])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 4